In [ ]:
import nexradaws
import tempfile
import os
import shutil
import pyart
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import cartopy.crs as ccrs


import pytz

In [ ]:
templocation = tempfile.mkdtemp()
conn = nexradaws.NexradAwsInterface()
scans = conn.get_avail_scans(2020, 8, 10,'KLOT')

In [ ]:
lcn = templocation
localfiles = conn.download(scans[10:20],lcn)

In [ ]:
radar = pyart.io.read(localfiles.success[0].filepath)


In [ ]:
myf = plt.figure(figsize=[10,10])
myd = pyart.graph.RadarMapDisplay(radar)
myd.plot_ppi_map('reflectivity', 0, vmin=-8, vmax=64)

In [ ]:
a=scans[-1]

In [ ]:
a.scan_time

In [ ]:
#https://stackoverflow.com/questions/32237862/find-the-closest-date-to-a-given-date/32237949
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [ ]:
times = [scan.scan_time for scan in scans]

#Need to clean
good_scans = []
good_times = []
for i in range(len(scans)):
    if times[i] is not None:
        good_times.append(times[i])
        good_scans.append(scans[i])
        
nearest_time = nearest(good_times, datetime(2020,8,10,20,6,55,0, pytz.UTC))


index = times.index(nearest_time)

In [ ]:
times

In [ ]:
good_times[index]

In [ ]:
def get_my_radar(connex, site, this_datetime):
    tlocation = tempfile.mkdtemp()
    these_scans = connex.get_avail_scans(this_datetime.year,this_datetime.month, this_datetime.day, site)
    these_times = [scan.scan_time for scan in these_scans]
    targ = this_datetime
    
    #Need to clean
    these_good_scans = []
    these_good_times = []
    for i in range(len(these_scans)):
        if these_times[i] is not None:
            these_good_times.append(these_times[i])
            these_good_scans.append(these_scans[i])
    
    print(len(these_good_scans), len(these_good_times))

    this_nearest_time = nearest(these_good_times, targ)
    this_index = these_good_times.index(this_nearest_time)
    lcn = templocation
    localfiles = conn.download(these_good_scans[this_index],lcn)
    return pyart.io.read(localfiles.success[0].filepath)

In [ ]:
snow_time_n = datetime(2021, 2, 15, 20, 6, 55, 0)
snow_time = snow_time_n.replace(tzinfo=pytz.UTC)

boom_time_n = datetime(2011, 5, 20, 11, 6, 55, 0)
boom_time = boom_time_n.replace(tzinfo=pytz.UTC)

hurricane_time_n = datetime(2017, 9, 6, 21, 48, 0)
hurricane_time = hurricane_time_n.replace(tzinfo=pytz.UTC)

der_time_n = datetime(2020, 8, 10, 20, 48, 0)
der_time = der_time_n.replace(tzinfo=pytz.UTC)

recent_time_n = datetime(2021, 2, 15, 21, 38, 0)
recent_time = datetime(2021, 2, 15, 21, 38, 0).replace(tzinfo=pytz.UTC)



radar = get_my_radar(conn, 'KEVX', recent_time)


In [ ]:
myf = plt.figure(figsize=[10,10])
myd = pyart.graph.RadarMapDisplay(radar)
myd.plot_ppi_map('reflectivity', 0, vmin=-8, vmax=64)

## So back to gridding

In [ ]:
boom_time = datetime(2011, 5, 20, 11, 6, 55, 0, pytz.UTC)

KVNX = get_my_radar(conn, 'KVNX', boom_time)
KTLX = get_my_radar(conn, 'KTLX', boom_time)
KINX = get_my_radar(conn, 'KINX', boom_time)


In [ ]:
grids = pyart.map.grid_from_radars((KINX, KVNX, KTLX),(16,401,401),
                   ((0.,15000.),(-400000.,400000.),(-400000.,400000.)),
                   refl_field='reflectivity', weighting_function='Barnes2')

In [ ]:

xgrids = grids.to_xarray()
box = (xgrids.lon.values.min(),xgrids.lon.values.max(), 
               xgrids.lat.values.min(), xgrids.lat.values.max())

In [ ]:
box

In [ ]:
print(grids.projection)
lon_0 = grids.get_projparams()['lon_0']
lat_0 = grids.get_projparams()['lat_0']
proj = ccrs.AzimuthalEquidistant(central_latitude=lat_0, central_longitude=lon_0)

In [ ]:
fig = plt.figure(figsize=[15, 7])
ax = plt.axes(projection=proj)
pc = xgrids.reflectivity.sel(z=1000, 
                        time=xgrids.time[0]).plot.pcolormesh(cmap=pyart.graph.cm_colorblind.HomeyerRainbow, 
                                                             vmin=-6, vmax=64)
gl = ax.gridlines(draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')

## Now lets use MetPy!

In [ ]:

import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Notice that we will use "plt" to access matplotlib
import matplotlib.pyplot as plt

import metpy.calc as mpcalc
import metpy.plots as mpplots

import numpy as np

from matplotlib.patheffects import withStroke
from metpy.io import parse_metar_file
from metpy.units import pandas_dataframe_to_unit_arrays

# Here is where we import the TDSCatalog class from siphon for obtaining our data 
from siphon.catalog import TDSCatalog

In [ ]:
metpy_time = recent_time
metpy_time_n = recent_time_n
# https://www.spc.noaa.gov/climo/reports/210215_rpts.html

KEOX = get_my_radar(conn, 'KEOX', metpy_time)
KEVX = get_my_radar(conn, 'KEVX', metpy_time)
KTLH = get_my_radar(conn, 'KTLH', metpy_time)
KMXX = get_my_radar(conn, 'KMXX', metpy_time)



In [ ]:
grids = pyart.map.grid_from_radars((KEOX, KEVX, KTLH, KMXX),(16,401,401),
                   ((0.,15000.),(-400000.,400000.),(-400000.,400000.)),
                   refl_field='reflectivity', weighting_function='Barnes2')


In [ ]:
xgrids = grids.to_xarray()
lon_0 = grids.get_projparams()['lon_0']
lat_0 = grids.get_projparams()['lat_0']
proj = ccrs.AzimuthalEquidistant(central_latitude=lat_0, central_longitude=lon_0)
box = (xgrids.lon.values.min(),xgrids.lon.values.max(), 
               xgrids.lat.values.min(), xgrids.lat.values.max())

In [ ]:
# METAR information is stored in a different 
# location from the previous THREDDS catalog, 
# notice the change in URL.
metar_cat = TDSCatalog('https://thredds-test.unidata.ucar.edu/thredds/catalog/noaaport/text/metar/catalog.xml')

# Open the metar file that contains data
# closest to the satellite image time, dt
#metar_text = metar_cat.datasets.filter_time_nearest(metpy_time_n).remote_open(mode='t')
metar_text = metar_cat.datasets.filter_time_range(metpy_time_n, metpy_time_n + timedelta(hours=1))[0].remote_open(mode='t')

In [ ]:
metar_text

In [ ]:
# parse_metar_file() outputs a pandas DataFrame
sfc_data = parse_metar_file(metar_text, year=metpy_time_n.year, month=metpy_time_n.month)

# Save the units for all columns to a new variable
sfc_units = sfc_data.units

# Filter out missing lat/lon data
sfc_data = sfc_data[sfc_data.latitude.notna() & sfc_data.longitude.notna()]

# Set missing weather condition data to an empty string, ''
sfc_data['current_wx1'][sfc_data['current_wx1'].isna()] = ''

In [ ]:
sfc_data['date_time'][100]

In [ ]:
# Create final data structure
#locs = proj.transform_points(ccrs.PlateCarree(), sfc_data['longitude'].m, sfc_data['latitude'].m)

#plot_mask = mpcalc.reduce_point_density(locs[..., :2], 175000)
sfc_data = pandas_dataframe_to_unit_arrays(sfc_data, sfc_units)

In [ ]:
sfc_data['wx_code'] = mpplots.wx_code_to_numeric(sfc_data['current_wx1'])
sfc_data['u'], sfc_data['v'] = mpcalc.wind_components(sfc_data['wind_speed'], sfc_data['wind_direction'])

plot_mask = mpcalc.reduce_point_density(locs[..., :2], 175000)

# Start by creating the matplotlib axes
fig = plt.figure(figsize=[15, 15])
ax = plt.axes(projection=proj)
pc = xgrids.reflectivity.sel(z=1000, 
                        time=xgrids.time[0]).plot.pcolormesh(cmap=pyart.graph.cm_colorblind.HomeyerRainbow, 
                                                             vmin=-6, vmax=64)
gl = ax.gridlines(draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
# Create the station plot object, stn,
# from the StationPlot class, using the
# PlateCarree projection
stn = mpplots.StationPlot(ax, sfc_data['longitude'].m, sfc_data['latitude'].m, transform=ccrs.PlateCarree(),
                         clip_on=True)

# Populate the temperature and dewpoint
stn.plot_parameter('NW', sfc_data['air_temperature'], color='red')
stn.plot_parameter('SW', sfc_data['dew_point_temperature'], color='blue')

# Populate the center circle cloud coverage and weather code
stn.plot_symbol('C', sfc_data['cloud_coverage'], mpplots.sky_cover)
stn.plot_symbol('E', sfc_data['wx_code'], mpplots.current_weather, color='blue')

# Populate the wind bard
stn.plot_barb(sfc_data['u'], sfc_data['v'])


ax.coastlines()
ax.add_feature(cfeature.LAKES, zorder=0)

gl = ax.gridlines(draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
ax.set_extent(box)